In [26]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Activation, Dropout, Flatten, GlobalAveragePooling2D
from keras import optimizers
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16

In [27]:
IMG_SIZE = 150
LEARNING_RATE = 0.001
EPOCHS = 5
BATCH_SIZE = 10

TRAIN_SIZE = 2000
TEST_SIZE = 250

TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'

In [28]:
datagen = ImageDataGenerator(rescale=1./255)

# augmented_gen = ImageDataGenerator(
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True)

train_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=['dogs', 'cats'],
    batch_size=BATCH_SIZE)

test_gen = datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=['dogs', 'cats'],
    batch_size=BATCH_SIZE)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [29]:
# Initialize VGG16 Model
vgg16_model = VGG16(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [30]:
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

# Only allow training on top layers
for layer in model.layers:
    layer.trainable = False
    
# Add top layers
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
    
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [31]:
# Begin training (Rocky montage music - 'Getting stronga!')
training = model.fit_generator(
    generator=train_gen,
    steps_per_epoch= TRAIN_SIZE // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_gen,
    validation_steps= TEST_SIZE // BATCH_SIZE)

Epoch 1/5
200/200 [==============================] - 8s 41ms/step - loss: 0.5171 - acc: 0.7500 - val_loss: 0.3602 - val_acc: 0.9000
Epoch 2/5
200/200 [==============================] - 8s 40ms/step - loss: 0.3840 - acc: 0.8280 - val_loss: 0.2585 - val_acc: 0.9000
Epoch 3/5
200/200 [==============================] - 8s 42ms/step - loss: 0.3961 - acc: 0.8220 - val_loss: 0.2583 - val_acc: 0.9120
Epoch 4/5
200/200 [==============================] - 8s 41ms/step - loss: 0.3344 - acc: 0.8510 - val_loss: 0.2399 - val_acc: 0.9200
Epoch 5/5
200/200 [==============================] - 8s 38ms/step - loss: 0.3333 - acc: 0.8720 - val_loss: 0.2065 - val_acc: 0.9080


In [32]:
model.save('models/vgg16-150.h5')